In [1]:
import pandas as pd
import psycopg2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
CONNECT_DB = "host='localhost' port='5432' dbname='banksim_artificial' user='cloud_user' password='cloud_user'"

kesamaaan table dan value dari kedua table
- source = customer
- target = merchant

In [3]:
# query for create table tips
def create_new_table(create_table_query):
    # cxn=None
    try:
        # Make connection to db
        cxn = psycopg2.connect(CONNECT_DB)

        # Create a cursor to db
        cur = cxn.cursor()
        
        # Send sql query to request
        cur.execute(create_table_query)
        records = cxn.commit()
    except (Exception, psycopg2.Error) as error :
        print ("Error while connecting to PostgreSQL", error)
    # except is when query failed run because failed to connect the postgresql

    finally:
        #closing database connection.
        if(cxn):
            cur.close()
            cxn.close()
            print("PostgreSQL connection is closed")
        elif(error == 'relation "tips" already exists'):
            cur.close()
            cxn.close()
            records = error

    # di relational database jumlah koneksi dibatasi, maka tidak bisa dibuka terus-menerus
    # begitu query selesai, koneksi harus di tutup supaya tidak overqueue
    print(f'Records:\n {records}')

In [16]:
query_first_table = '''CREATE TABLE bsNet140513 (
	Source VARCHAR(20),
	Target VARCHAR(20),
	Weight NUMERIC(10,2),
	typeTrans VARCHAR(50),
	fraud INTEGER
);'''

first_table = create_new_table(query_first_table)

PostgreSQL connection is closed
Records:
 None


In [5]:
query_second_table = '''CREATE TABLE bs140513(
	step INTEGER,
	customer VARCHAR(20),
	age VARCHAR(2),
	gender VARCHAR(1),
	zipcodeOri VARCHAR(10),
	merchant VARCHAR(20),
	zipMerchant VARCHAR(10),
	category VARCHAR(50),
	amount NUMERIC(10,2),
	fraud INTEGER
);'''

second_table = create_new_table(query_second_table)

PostgreSQL connection is closed
Records:
 None


In [6]:
# insert_data_table('table_data.csv', 'table_ondb')
def insert_data_table(data_csv, table_in_database):
    try:
        # Make connection to db
        cxn = psycopg2.connect(CONNECT_DB)
        
        # Create a cursor to db
        cur = cxn.cursor()
        
        # open csv file on tips.csv to process the csv file
        with open(data_csv, 'r') as f:
            # skip first row, header row
            next(f)
            cur.copy_from(f, table_in_database, sep=",")
            cxn.commit()

    except (Exception, psycopg2.Error) as error :
        print ("Error while connecting to PostgreSQL", error)
        
    finally:
        #closing database connection.
        if(cxn):
            cur.close()
            cxn.close()
            print("PostgreSQL connection is closed")

- DATABASE `banksim_artificial`
- USER `cloud_user`
- PASSWORD `'cloud_user'`

`GRANT ALL PRIVILEGES ON DATABASE banksim_artificial TO cloud_user;`

`\c banksim_artificial postgres`

`GRANT ALL ON SCHEMA public TO cloud_user;`

In [17]:
insert_data_table('bs140513_032310.csv','bs140513')
insert_data_table('bsNET140513_032310.csv', 'bsnet140513')

PostgreSQL connection is closed


In [7]:
def fetch_query(sql_query):
    try:
        # connect to db
        conn = psycopg2.connect(CONNECT_DB)
        # connect cursor to db
        curs = conn.cursor()
        # send postgresql query to request
        curs.execute(sql_query)
        records = curs.fetchall()

    except(Exception, psycopg2.Error) as error:
        print ("Error while connecting to PostgreSQL", error)
    
    # close the connection
    finally:
        if(conn):
            curs.close()
            conn.close()
            # print('PostgreSQL Connection is Closed')
        return records


In [10]:
print('bs140513 table rows :', fetch_query('SELECT COUNT(*) FROM bs140513'))
print('bsnet140513 table rows :', fetch_query('SELECT COUNT(*) FROM bsnet140513'))

bs140513 table rows : [(594643,)]
bsnet140513 table rows : [(594643,)]


In [17]:
fetch_query('SELECT * FROM bsnet140513 LIMIT 5')

[("'C1093826151'", "'M348934600'", Decimal('4.55'), "'es_transportation'", 0),
 ("'C352968107'", "'M348934600'", Decimal('39.68'), "'es_transportation'", 0),
 ("'C2054744914'",
  "'M1823072687'",
  Decimal('26.89'),
  "'es_transportation'",
  0),
 ("'C1760612790'", "'M348934600'", Decimal('17.25'), "'es_transportation'", 0),
 ("'C757503768'", "'M348934600'", Decimal('35.72'), "'es_transportation'", 0)]

In [16]:
fetch_query('''SELECT * FROM bs140513 LIMIT 3''')

[(0,
  "'C1093826151'",
  "'4'",
  "'M'",
  "'28007'",
  "'M348934600'",
  "'28007'",
  "'es_transportation'",
  Decimal('4.55'),
  0),
 (0,
  "'C352968107'",
  "'2'",
  "'M'",
  "'28007'",
  "'M348934600'",
  "'28007'",
  "'es_transportation'",
  Decimal('39.68'),
  0),
 (0,
  "'C2054744914'",
  "'4'",
  "'F'",
  "'28007'",
  "'M1823072687'",
  "'28007'",
  "'es_transportation'",
  Decimal('26.89'),
  0)]

In [21]:
def pandas_db_server_fetch(sql_query):
    try:
        # Make connection to db
        conn = psycopg2.connect(CONNECT_DB)

        # Send sql query to request and create dataframe
        df = pd.read_sql(sql_query, conn)

    except (Exception, psycopg2.Error) as error :
        print ("Error while connecting to PostgreSQL", error)

    finally:
        #closing database connection.
        if(conn):
            conn.close()
            # print("PostgreSQL connection is closed")
        return df

menjalankan function string replace dengan regex r `str.replace(r"'","")` untuk menghapus kutip di dalam value pada kolom dengan

In [52]:
# mengambil table dari sql dan ditampung di dataframe dengan pandas
df_bs140513 = pandas_db_server_fetch('SELECT * FROM bs140513')

# menghilangkan kutip pada value di dalam kolom dengan str replace
for col_string in ['customer', 'age', 'gender', 'zipcodeori','merchant','zipmerchant','category']:
    df_bs140513[col_string] = df_bs140513[col_string].str.replace(r"'", "")
df_bs140513.head()

,step,customer,age,gender,zipcodeori,merchant,zipmerchant,category,amount,fraud
0,0,C1093826151,4,M,28007,M348934600,28007,es_transportation,4.55,0
1,0,C352968107,2,M,28007,M348934600,28007,es_transportation,39.68,0
2,0,C2054744914,4,F,28007,M1823072687,28007,es_transportation,26.89,0
3,0,C1760612790,3,M,28007,M348934600,28007,es_transportation,17.25,0
4,0,C757503768,5,M,28007,M348934600,28007,es_transportation,35.72,0


In [34]:
print(df_bs140513.shape)
df_bs140513.info()

(594643, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594643 entries, 0 to 594642
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   step         594643 non-null  int64  
 1   customer     594643 non-null  object 
 2   age          594643 non-null  object 
 3   gender       594643 non-null  object 
 4   zipcodeori   594643 non-null  object 
 5   merchant     594643 non-null  object 
 6   zipmerchant  594643 non-null  object 
 7   category     594643 non-null  object 
 8   amount       594643 non-null  float64
 9   fraud        594643 non-null  int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 45.4+ MB


In [36]:
df_bs140513.isnull().sum()

step           0
customer       0
age            0
gender         0
zipcodeori     0
merchant       0
zipmerchant    0
category       0
amount         0
fraud          0
dtype: int64

In [38]:
display(df_bs140513.describe())
display(df_bs140513.describe(include=[object]))

,step,amount,fraud
count,594643.000000,594643.000000,594643.000000
mean,94.986827,37.890135,0.012108
std,51.053632,111.402831,0.109369
min,0.000000,0.000000,0.000000
25%,52.000000,13.740000,0.000000
50%,97.000000,26.900000,0.000000
75%,139.000000,42.540000,0.000000
max,179.000000,8329.960000,1.000000


,customer,age,gender,zipcodeori,merchant,zipmerchant,category
count,594643,594643,594643,594643,594643,594643,594643
unique,4112,8,4,1,50,1,15
top,C1978250683,2,F,28007,M1823072687,28007,es_transportation
freq,265,187310,324565,594643,299693,594643,505119


melihat masing-masing unique value dengan panjang jumlah variabel unique nya masing-masing

In [41]:
for cat_col in ['age','gender','category']:
    print(f'{cat_col}: {df_bs140513[cat_col].unique()}')
    print(f'length of {cat_col} : {len(df_bs140513[cat_col].unique())}\n')

age: ['4' '2' '3' '5' '1' '6' 'U' '0']
length of age : 8

gender: ['M' 'F' 'E' 'U']
length of gender : 4

category: ['es_transportation' 'es_health' 'es_otherservices' 'es_food'
 'es_hotelservices' 'es_barsandrestaurants' 'es_tech' 'es_sportsandtoys'
 'es_wellnessandbeauty' 'es_hyper' 'es_fashion' 'es_home' 'es_contents'
 'es_travel' 'es_leisure']
length of category : 15



menampilkan data `bs140513` dengan `age` yang hanya berisi `U`

In [53]:
df_bs140513[df_bs140513['age'] == 'U'] 

,step,customer,age,gender,zipcodeori,merchant,zipmerchant,category,amount,fraud
467,0,C2075935351,U,E,28007,M1823072687,28007,es_transportation,21.25,0
1494,0,C914000857,U,E,28007,M348934600,28007,es_transportation,35.59,0
1856,0,C808326652,U,E,28007,M348934600,28007,es_transportation,56.78,0
2092,0,C1374607221,U,E,28007,M1823072687,28007,es_transportation,17.47,0
2796,1,C1871125244,U,E,28007,M547558035,28007,es_fashion,40.38,0
...,...,...,...,...,...,...,...,...,...,...
592375,179,C1215440829,U,E,28007,M1823072687,28007,es_transportation,44.16,0
592594,179,C2075935351,U,E,28007,M1823072687,28007,es_transportation,16.78,0
593660,179,C1871125244,U,E,28007,M1823072687,28007,es_transportation,56.11,0
594128,179,C1374607221,U,E,28007,M1823072687,28007,es_transportation,4.44,0


melihat masing-masing unique value dari kolom `age`, `gender`, dan `category` dengan method `value_counts()` dari pandas

In [46]:
for cat_col in ['age','gender','category']:
    print(df_bs140513[cat_col].value_counts(),'\n')

2    187310
3    147131
4    109025
5     62642
1     58131
6     26774
0      2452
U      1178
Name: age, dtype: int64 

F    324565
M    268385
E      1178
U       515
Name: gender, dtype: int64 

es_transportation        505119
es_food                   26254
es_health                 16133
es_wellnessandbeauty      15086
es_fashion                 6454
es_barsandrestaurants      6373
es_hyper                   6098
es_sportsandtoys           4002
es_tech                    2370
es_home                    1986
es_hotelservices           1744
es_otherservices            912
es_contents                 885
es_travel                   728
es_leisure                  499
Name: category, dtype: int64 



menjalankan function string replace dengan regex r `str.replace(r"'","")` untuk menghapus kutip di dalam value pada kolom dengan

In [30]:
# mengambil table dari sql dan ditampung di dataframe dengan pandas
df_bsnet140513 = pandas_db_server_fetch('SELECT * FROM bsnet140513')

# menghilangkan kutip pada value di dalam kolom dengan str replace
for col_string in ['source', 'target', 'typetrans']:
    df_bsnet140513[col_string] = df_bsnet140513[col_string].str.replace(r"'", "")
df_bsnet140513.head()

,source,target,weight,typetrans,fraud
0,C1093826151,M348934600,4.55,es_transportation,0
1,C352968107,M348934600,39.68,es_transportation,0
2,C2054744914,M1823072687,26.89,es_transportation,0
3,C1760612790,M348934600,17.25,es_transportation,0
4,C757503768,M348934600,35.72,es_transportation,0


In [35]:
print(df_bsnet140513.shape)
df_bsnet140513.info()

(594643, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594643 entries, 0 to 594642
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   source     594643 non-null  object 
 1   target     594643 non-null  object 
 2   weight     594643 non-null  float64
 3   typetrans  594643 non-null  object 
 4   fraud      594643 non-null  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 22.7+ MB


In [37]:
df_bsnet140513.isnull().sum()

source       0
target       0
weight       0
typetrans    0
fraud        0
dtype: int64

In [39]:
display(df_bsnet140513.describe())
display(df_bsnet140513.describe(include=[object]))

,weight,fraud
count,594643.000000,594643.000000
mean,37.890135,0.012108
std,111.402831,0.109369
min,0.000000,0.000000
25%,13.740000,0.000000
50%,26.900000,0.000000
75%,42.540000,0.000000
max,8329.960000,1.000000


,source,target,typetrans
count,594643,594643,594643
unique,4112,50,15
top,C1978250683,M1823072687,es_transportation
freq,265,299693,505119


melihat unique values dari kolom `typetrans` dengan jumlah unique valuesnya 

In [48]:
print('typetrans unique values: ', df_bsnet140513['typetrans'].unique())
print('length of typetrans : ', len(df_bsnet140513['typetrans'].unique()))

typetrans unique values:  ['es_transportation' 'es_health' 'es_otherservices' 'es_food'
 'es_hotelservices' 'es_barsandrestaurants' 'es_tech' 'es_sportsandtoys'
 'es_wellnessandbeauty' 'es_hyper' 'es_fashion' 'es_home' 'es_contents'
 'es_travel' 'es_leisure']
length of typetrans :  15


melihat masing-masing jumlah frequensi dari unique value pada `typetrans`

In [49]:
df_bsnet140513['typetrans'].value_counts()

es_transportation        505119
es_food                   26254
es_health                 16133
es_wellnessandbeauty      15086
es_fashion                 6454
es_barsandrestaurants      6373
es_hyper                   6098
es_sportsandtoys           4002
es_tech                    2370
es_home                    1986
es_hotelservices           1744
es_otherservices            912
es_contents                 885
es_travel                   728
es_leisure                  499
Name: typetrans, dtype: int64

Filtering Data on `bs140513` karena nilainya yang anomali
- Exclude Gender E and U , untuk memfilter gender yang berisi `E` dan `U` karena anomali
- Exclude Age U, untuk filter data yang `age`nya bukan `U` karena data `U` merupakan anomali

diakhiri dengan membuat table baru `bs140513` versi bersihnya

In [62]:
df_bs140513_clean = df_bs140513[~df_bs140513['gender'].isin(['E','U'])]
df_bs140513_clean = df_bs140513_clean[df_bs140513['age']!='U']

`age` diubah dari object ke integer karena seharusnya kolom angka menggunakan `astype(str).astype(int)`

In [66]:
# mengubah tipe data dari string object ke int
df_bs140513_clean['age'] = df_bs140513_clean['age'].astype(str).astype(int)

In [67]:
df_bs140513_clean['gender'].value_counts()

F    324565
M    268385
Name: gender, dtype: int64

In [68]:
display(df_bs140513_clean.describe())
display(df_bs140513_clean.describe(include=[object]))

,step,age,amount,fraud
count,592950.000000,592950.000000,592950.000000,592950.000000
mean,94.982803,3.007122,37.898186,0.012131
std,51.056692,1.325356,111.530644,0.109470
min,0.000000,0.000000,0.000000,0.000000
25%,52.000000,2.000000,13.750000,0.000000
50%,97.000000,3.000000,26.900000,0.000000
75%,139.000000,4.000000,42.540000,0.000000
max,179.000000,6.000000,8329.960000,1.000000


,customer,gender,zipcodeori,merchant,zipmerchant,category
count,592950,592950,592950,592950,592950,592950
unique,4100,2,1,50,1,15
top,C1978250683,F,28007,M1823072687,28007,es_transportation
freq,265,324565,592950,298771,592950,503699


membuat table baru versi bersih `bsnet140513` dengan memilih rows yang customernya hanya terdapat di `bs140513`

In [79]:
# menggunakan isin namun menggunakan table berbeda yang berisi value serupa
df_bsnet140513_clean = df_bsnet140513[df_bsnet140513['source'].isin(df_bs140513_clean['customer'])]

In [80]:
df_bsnet140513_clean.shape

(592950, 5)

In [73]:
df_bs140513_clean.shape

(592950, 10)

membuat kolom baru `fraud status`, untuk mengisi status sesuai `0` jadi `no` dan `1` jadi `yes`

In [81]:
# membuat kolom baru fraud status yang mengganti nilai 0 menjadi No dengan mapping dictionary
df_bsnet140513_clean['fraudstatus'] = df_bsnet140513_clean['fraud'].map({0:'No', 1:'Yes'})

C:\Users\Hendry\AppData\Local\Temp\ipykernel_8768\2492722945.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bsnet140513_clean['fraudstatus'] = df_bsnet140513_clean['fraud'].map({0:'No', 1:'Yes'})


In [82]:
df_bsnet140513_clean.head()

,source,target,weight,typetrans,fraud,fraudstatus
0,C1093826151,M348934600,4.55,es_transportation,0,No
1,C352968107,M348934600,39.68,es_transportation,0,No
2,C2054744914,M1823072687,26.89,es_transportation,0,No
3,C1760612790,M348934600,17.25,es_transportation,0,No
4,C757503768,M348934600,35.72,es_transportation,0,No


In [83]:
df_bsnet140513_clean['fraudstatus'].value_counts()

No     585757
Yes      7193
Name: fraudstatus, dtype: int64

membuat table baru `bsnet140513` versi bersih dengan kolom `fraud` yang menjadi `yes` dan `no` 

In [86]:
df_bsnet140513_clean = df_bsnet140513_clean.loc[:,df_bsnet140513_clean.columns!='fraud']
df_bsnet140513_clean.columns = ['source', 'target', 'weight', 'typetrans', 'fraud']
df_bsnet140513_clean.head()

,source,target,weight,typetrans,fraud
0,C1093826151,M348934600,4.55,es_transportation,No
1,C352968107,M348934600,39.68,es_transportation,No
2,C2054744914,M1823072687,26.89,es_transportation,No
3,C1760612790,M348934600,17.25,es_transportation,No
4,C757503768,M348934600,35.72,es_transportation,No


pada table `bs140513` versi bersihnya mengganti penamaan value dari 
- `gender` dari `M` menjadi `Male` dan `F` menjadi `Female`
- `fraud` dari `0` menjadi `no` dan `1` menjadi `yes`

In [89]:
df_bs140513_clean.replace({'gender':{'M':'Male', 'F':'Female'}, 
                           'fraud':{0:'No', 1:'Yes'}}, inplace=True)

In [90]:
df_bs140513_clean.head()

,step,customer,age,gender,zipcodeori,merchant,zipmerchant,category,amount,fraud
0,0,C1093826151,4,Male,28007,M348934600,28007,es_transportation,4.55,No
1,0,C352968107,2,Male,28007,M348934600,28007,es_transportation,39.68,No
2,0,C2054744914,4,Female,28007,M1823072687,28007,es_transportation,26.89,No
3,0,C1760612790,3,Male,28007,M348934600,28007,es_transportation,17.25,No
4,0,C757503768,5,Male,28007,M348934600,28007,es_transportation,35.72,No


Export data ke csv

In [93]:
df_bs140513_clean.to_csv('bs140513_clean.csv',index=False, header=False)
df_bsnet140513_clean.to_csv('bsNET140513_clean.csv',index=False, header=False)

query untuk tabel versi bersihnya

CREATE TABLE bsNet140513_clean (
	Source VARCHAR(20),
	Target VARCHAR(20),
	Weight NUMERIC(10,2),
	typeTrans VARCHAR(50),
	fraud VARCHAR(4)
);

CREATE TABLE bs140513_clean (
	step INTEGER,
	customer VARCHAR(20),
	age INTEGER,
	gender VARCHAR(10),
	zipcodeOri VARCHAR(10),
	merchant VARCHAR(20),
	zipMerchant VARCHAR(10),
	category VARCHAR(50),
	amount NUMERIC(10,2),
	fraud VARCHAR(4)
);